I trained this model but did not use it because the model is trained with the previous 10 months' information so it does not go further than predicting the accident number on 2021-01, because I have no information that I can provide to the model about the accidents in 2021, so I decided to change my strategy. But the model is saved under 'models\model1'.

In [1]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import seaborn as sns
import joblib

Filtering the data, these steps were explained in the `drawingGraph.ipynb` file

In [2]:
data =  pd.read_csv("monatszahlen2209_verkehrsunfaelle.csv")
data = data.iloc[:, :5] 
data.rename(columns={'MONATSZAHL': 'category'
                  , 'AUSPRÄGUNG': 'accidentType'
                  , 'JAHR': 'year'
                  , 'MONAT': 'month'
                  , 'WERT': 'value'}, inplace=True)
data = data[data["month"].str.lower() != "summe"]
data = data[data['accidentType'] == 'insgesamt']
data = data[data['category'] == 'Alkoholunfälle']
data['month'] = data['month'].apply(lambda x: int(str(x)[-2:]) if str(x).isdigit() else x)
toBeUsedForTesting = data #I will use this later for testing
data = data[data['year'] <= 2020].reset_index(drop=True) #because we will make predictions for 2021
data['Date'] = pd.to_datetime(data[['year', 'month']].assign(day=1)) #[day] is missing error was shown so I put the first day for each date 
data = data.sort_values('Date')

In [3]:
values = data['value'].values.reshape(-1, 1) # I will only get the values because we make the predictions accordingly, to make it 2d I reshaped it 
scaler = MinMaxScaler(feature_range=(-1, 1)) # I will use tanh so scaled it to (-1,1)
values = scaler.fit_transform(values)

In [4]:
# I split the dataset into train and test sets
train_size = int(len(values) * 0.80)
test_size = len(values) - train_size
train = values[0:train_size,:]
test = values[train_size:len(values),:]
print(len(train), len(test))

201 51


In [5]:
# I am creating a dataset with a memory so I will use it in the training
# The model takes the previous ten months while training 
def dataset_creation(data, pre_steps = 10):
    X, Y = [], []
    for i in range(len(data)-pre_steps):
        X.append(data[i:(i+pre_steps), 0])      # I get the past information
        Y.append(data[i + pre_steps, 0])        # I get the element to predict
    return np.array(X), np.array(Y)

# Creating train and test datasets
trainX, trainY = dataset_creation(train)
testX, testY = dataset_creation(test)

In [6]:
# We are transforming the input data into the shape that the LSTM model expects and by putting 1 we are combining previous 10 months information 
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
model = Sequential()
model.add(LSTM(4, input_shape=(1, 10)))     # LSTM with 4 neurons 
model.add(Dropout(0.2))                     # I realized overfitting so I put Dropout to stop it
model.add(Dense(1))                         # Creating the neural network layer
model.compile(loss='mean_squared_error', optimizer='adam')  # I chose MSE for loss calculation and adam optimizer for faster convergence
model.fit(trainX, trainY, epochs=1000, batch_size=4, verbose=2)     #Training the model


Epoch 1/1000
48/48 - 4s - loss: 0.1315 - 4s/epoch - 92ms/step
Epoch 2/1000
48/48 - 0s - loss: 0.0909 - 262ms/epoch - 5ms/step
Epoch 3/1000
48/48 - 0s - loss: 0.0771 - 139ms/epoch - 3ms/step
Epoch 4/1000
48/48 - 0s - loss: 0.0763 - 138ms/epoch - 3ms/step
Epoch 5/1000
48/48 - 0s - loss: 0.0736 - 140ms/epoch - 3ms/step
Epoch 6/1000
48/48 - 0s - loss: 0.0688 - 146ms/epoch - 3ms/step
Epoch 7/1000
48/48 - 0s - loss: 0.0715 - 168ms/epoch - 4ms/step
Epoch 8/1000
48/48 - 0s - loss: 0.0655 - 155ms/epoch - 3ms/step
Epoch 9/1000
48/48 - 0s - loss: 0.0585 - 155ms/epoch - 3ms/step
Epoch 10/1000
48/48 - 0s - loss: 0.0598 - 187ms/epoch - 4ms/step
Epoch 11/1000
48/48 - 0s - loss: 0.0543 - 147ms/epoch - 3ms/step
Epoch 12/1000
48/48 - 0s - loss: 0.0591 - 132ms/epoch - 3ms/step
Epoch 13/1000
48/48 - 0s - loss: 0.0548 - 139ms/epoch - 3ms/step
Epoch 14/1000
48/48 - 0s - loss: 0.0547 - 157ms/epoch - 3ms/step
Epoch 15/1000
48/48 - 0s - loss: 0.0518 - 153ms/epoch - 3ms/step
Epoch 16/1000
48/48 - 0s - loss: 0.0

In [7]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# I scaled the predictions before so now by using inverse sclaing I will return them into normal numbers 
# I did this because LSTM needs scaling to train the model
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1, 1))
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1, 1))
# I am making calculation looking at this value I tried to improve my model
from sklearn.metrics import mean_squared_error
trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

2/2 [==============================] - 0s 3ms/step
Train Score: 7.35 RMSE
Test Score: 8.17 RMSE


In [8]:
# Save the scaler and the model
model.save('models\model1\my_model.h5') 
joblib.dump(scaler, 'models\model1\my_scaler.pkl') 

['models\\model1\\my_scaler.pkl']

In [9]:
def predict_for_date(year, month, data):
    # I am taking the previous 10 months of the input data from the dataset
    start_date = pd.to_datetime("{}-{}-01".format(year if month > 10 else year-1, (month-10)%12))
    end_date = pd.to_datetime("{}-{}-01".format(year, month))
    data = data[(data['Date'] >= start_date) & (data['Date'] < end_date)]
    # Scaling these values to feed to my model
    values = data['value'].values.reshape(-1, 1)
    values = scaler.transform(values)
    values = np.reshape(values, (1, 1, 10)) # 10 previous months
    prediction = model.predict(values)
    prediction = scaler.inverse_transform(prediction) # I want to compare the data with the real data I got from the dataset
    return prediction[0,0]

# Use the function to predict for January 2021.
prediction = predict_for_date(2021, 1, data)
print('Prediction for January 2021: ', prediction)

temp = toBeUsedForTesting[toBeUsedForTesting['year']==2021]
temp = temp[temp['month']==1]
temp = temp['value'].iloc[0]
print('Real Value: ', temp)



1/1 [==============================] - 0s 26ms/step
Prediction for January 2021:  22.89907
Real Value:  16.0
